# Домашнее задание 2 по теме: "Извлечение коллокаций + NER"

Я использовал данные из категории **"Cell Phones & Accessories"** из предложенного датасета при подготовке домашнего задания.  

Как можно найти упоминания товаров в отзывах (без использования обучающего набора):
1. rule-based системы, которые помогут найти сущности по шаблону (для английского языка см., например, [spaCy](https://spacy.io/usage/rule-based-matching)); этот подход требует подготовки некоторого небольшого словаря типов товаров (например, *телефон*, *смартфон*, *сотовый* и др.), данные можно взять из Metadata; основным его минусом является ограниченность его применения (например, мы не можем за два шага расширить домен), также такой метод плох для сложных случаев, о которых не догадывался создатель; плюсом же является то, что точность (precision) такого решения будет очень высокой, ибо всё, что задумывал автор такой системы будет обрабатываться почти идеально;
2. чуть больше расширить и частично автоматизировать описанный выше подход можно с помощью сгенерированного словаря на основе эмбеддингов синонимов (`most_similar`); особого успеха система может добиться, если в ней будет использоваться модель дистрибутивной семантики, способная работать с подсловами (это может помочь решить проблему с неизвестными словами); проблемой такой системы будут False Positives, которые понизят точность; достоинство системы заключается в уменьшении ручной работы и большем *обобщении*, если можно так сказать;
3. ещё одна система может быть построена на основе морфосинтаксических шаблонов, которые позволяют избежать некоторого мусора (*но могут и добавить*), так как ожидается, что сущности схожи по форме и синтаксической роли, однако так же, как и первая модель, могут плохо сработать в сложных случаях. 

Я при выполнении домашнего задания воспользовался всем по чуть-чуть: 1) составил словарь типов товаров на основе категорий релевантных товаров из метаданных; 2) раширил этот словарь посредством модели дистрибутивной семантики; 3) для определения сущностей учитывал синтаксический шаблон NP. NB! Хочу сразу заметить, что моя система неидеальна, однако она может быть применима ко всей категории, а не к какому-то отдельному типу товара, и почти не требует ручного вмешательства.

In [278]:
import pandas as pd
import json
from tqdm import tqdm
import spacy
from string import punctuation
import re
from collections import Counter
import numpy as np
from gensim.models import KeyedVectors
from nltk import word_tokenize
from nltk.collocations import *
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

Некоторые предварительные процессы (инициализация моделей, подготовка `tqdm`).

In [10]:
tqdm.pandas()
nlp = spacy.load("en_core_web_sm")
wv = KeyedVectors.load_word2vec_format('wiki-news-300d-1M-subword.vec')

## Подготовка датасетов

Основной датасет с отзывами. Удаляем лишнее и собираем список уникальных `id`товаров.

In [12]:
with open('Cell_Phones_and_Accessories_5.json') as f:
    data = f.readlines()

In [13]:
dataset = []
for d in tqdm(data):
    dataset.append(json.loads(d))
dataset = pd.DataFrame(dataset)
dataset = dataset.drop(['overall', 'verified', 'reviewTime',
                       'style', 'reviewerName', 'summary',
                       'unixReviewTime', 'vote', 'image', 'reviewerID'], axis=1)
dataset = dataset.drop_duplicates()

ids_products = list(set(dataset['asin'].tolist()))
print('всего уникальных товаров в датасете:', len(ids_products))

# dataset.to_csv('dataset.csv', index=False)

100%|██████████████████████████████| 1128437/1128437 [00:11<00:00, 94499.24it/s]


всего уникальных товаров в датасете: 48186


In [85]:
dataset

,asin,reviewText
0,7508492919,Looks even better in person. Be careful to not...
1,7508492919,When you don't want to spend a whole lot of ca...
2,7508492919,"so the case came on time, i love the design. I..."
3,7508492919,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...
4,7508492919,"I liked it because it was cute, but the studs ..."
...,...,...
1128432,B01HJC7N4C,Good for viewing. But doesn't have a button or...
1128433,B01HJC7N4C,I was given the Rockrok 3D VR Glasses Headset ...
1128434,B01HJC7N4C,Super Fun! The RockRoc 3d vr headset is waaaay...
1128435,B01HJC7N4C,Love it!\n\nI've had other VR glasses which al...


Датасет метаданных. Удаляем лишнее.

In [15]:
with open('meta_Cell_Phones_and_Accessories.json') as f:
    METAdata = f.readlines()

METAdataset = []
for d in tqdm(METAdata):
    METAdataset.append(json.loads(d))
METAdataset = pd.DataFrame(METAdataset).drop_duplicates(subset=['asin'])
METAdataset = METAdataset.drop(['tech1', 'description', 'fit', 'also_buy',
                               'tech2', 'feature', 'rank', 'also_view',
                               'details', 'similar_item', 'date', 'price',
                               'imageURL', 'imageURLHighRes'], axis=1)

# METAdataset.to_csv('METAdataset.csv', index=False)

100%|█████████████████████████████████| 590071/590071 [02:10<00:00, 4519.73it/s]


## Подготовка словаря

Выбираем только те товары, на которые в основном датасете есть отзывы.

In [16]:
METAdataset = METAdataset[METAdataset['asin'].isin(ids_products)]

In [17]:
punctuation = '[{}]'.format(punctuation)

Функция для сбора типов товаров из названий категорий и их лемматизации (это необходимо, так как они почти всегда во множественном числе).

In [18]:
def get_product_types(categories):
    types = []

    for cat in tqdm(categories):
        some_types = [w for w in re.split(punctuation, cat) if w != '']
        for t in some_types:
            doc = nlp(t.lower().strip())
            res = [token.lemma_ for token in doc]
            if len(res) > 1:
                col = ' '.join(res)
                types.append(col)
            try:
                types.append(res[len(res) - 1])
            except:
                pass
        
    return types

Фильтрую по частотности >= 10 (это нужно, так как и в дереве категорий в датасете много мусора).

In [19]:
all_categories = [c for cat in METAdataset['category'].to_list() for c in cat]
all_categories = [cat[0] for cat in Counter(all_categories).most_common() if cat[1] >= 10]
all_categories = get_product_types(all_categories)

100%|███████████████████████████████████████████| 51/51 [00:00<00:00, 55.40it/s]


С помощью spaСy будем доставать вершины из названий товаров (это дополнительное наполнение словаря типов товаров).

In [20]:
def get_head(name):
    doc = nlp(name)
    try:
        for chunk in doc.noun_chunks:
            head = chunk.root.text
            break
        return head
    except:
        return np.nan

In [21]:
METAdataset['head'] = METAdataset['title'].progress_apply(lambda t: get_head(t))
METAdataset = METAdataset.dropna(subset=['head'])

100%|████████████████████████████████████| 48172/48172 [05:39<00:00, 142.00it/s]


Эти типы товаров фильтруем по частотности >= 20, так как здесь ещё больше мусора.

In [22]:
types = [head[0].lower() for head in Counter(METAdataset['head'].to_list()).most_common() if \
         (head[1] >= 20) and (len(head[0]) > 3)]
types.extend(all_categories)
types = list(set(types))

In [23]:
types

['reiko',
 'blue',
 'holster',
 'sim card tool',
 'dock',
 'accessory kit',
 'car speakerphone',
 'adapter',
 'quick',
 'case',
 'nylon',
 'accessory',
 'desire',
 'phone charm',
 'protectors',
 'mybat',
 'silver',
 'shield',
 'import',
 'armband',
 'lightning',
 'broadband',
 'icarez',
 'replacement part',
 'shell',
 'signal booster',
 'cradle',
 'combo',
 'diamond',
 'door',
 'ixcc',
 'film',
 'edition',
 'mobile broadband',
 'eforcity&reg',
 'attachment',
 'snap',
 'nexus',
 'unlocked',
 'eforcity',
 'samsung',
 'dry bag',
 'earbuds',
 'tripod',
 'mat',
 'car charger',
 'glasses',
 'pink',
 'wallet',
 'etech',
 'series',
 'smartwatch accessory',
 'accessories',
 'touch',
 'huawei',
 'glass',
 'compatible',
 'portable power bank',
 'otterbox',
 'bold',
 'speaker',
 'car accessory',
 'sleeve',
 'mobile flash',
 'bank',
 'cord',
 'sync',
 'epartsolution',
 'earphones',
 'unlocked cell phone',
 'gear',
 'certified',
 'virtual reality headset',
 'hybrid',
 'power adapter',
 'wall charger

### Генерация дополнительных слов в словарь на основе модели дистрибутивной семантики FastText

Используется модель `wiki-news-300d-1M-subword.vec` с сайта [FastText](https://fasttext.cc/docs/en/english-vectors.html).  
Ниже приведён код в комментарии, который можно применить, чтобы не загружать модель и искать соседей, а сразу получить расширенный словарь.

In [2]:
wv = KeyedVectors.load_word2vec_format('wiki-news-300d-1M-subword.vec')

In [40]:
generated_types = []
for t in tqdm(types):
    generated_types.append(t)
    if len(t.split()) == 1:
        try:
            generated_types.extend([neighb[0].lower() for neighb in wv.most_similar(t)])
        except KeyError:
            pass
generated_types = list(set(generated_types))

100%|█████████████████████████████████████████| 223/223 [00:11<00:00, 19.89it/s]


In [43]:
print('размер словаря до генерации:', len(types))
print('размер словаря после генерации:', len(generated_types))

размер словаря до генерации: 223
размер словаря после генерации: 1604


Соответствующий файл приложен в репозитории.

In [388]:
# with open('generated_types.txt', 'w') as fh:
#     fh.writelines("%s\n" % g for g in generated_types)
    
# with open('generated_types.txt', 'r') as fh:
#     generated_types = fh.readlines()

Честно говоря, нужно было всё же брать меньше синонимов: в итоговом словаре есть явно лишние слова.

## Теггер

Теггер я написал вручную. Он работает следующим образом: 1) берёт NP; 2) если вершина в ней есть в словаре, до считает (пока что) всю NP за сущность; 3) (тут и начинается элемент rule-based) если в начале группы есть артикль, местоимение, числительное, наречие или прилагательное, то они "удаляются" из сущности (но только по-одному, так как прилагательные вполне могут быть и частью сущности); 4) далее идёт коротенькое правило по поводу цифр после вершины, которые не учитываются парсером spaCy.

In [176]:
def NER(review):
    NEs = []
    
    doc = nlp(review.lower())  
    for chunk in doc.noun_chunks:
        # NE = chunk.text
        head = chunk.root.text
        if head in generated_types:
            NE = chunk.text
            info = nlp(NE)
            negative = 0
            if len(NE.split()) > 1:
                try:
                    if info[negative].pos_ == 'DET':
                        negative += 1
                    if info[negative].pos_ == 'PRON':
                        negative += 1
                    if info[negative].pos_ == 'NUM':
                        negative += 1
                    if info[negative].pos_ == 'ADV':
                        negative += 1
                    if info[negative].pos_ == 'ADJ':
                        negative += 1
                except IndexError:
                    continue
            NE = ' '.join([token.text for token in info[negative:]])
            if NE == ' ' or '':
                continue
            tokenized = [token.text for token in doc]
            ind = tokenized.index(head)
            if ind != len(tokenized) - 1:
                potential_num = tokenized[tokenized.index(head) + 1]
                if re.search(r'\d+', potential_num) != None:
                    NEs.append('{} {}'.format(NE, potential_num))
                else:
                    NEs.append(NE)
            else:
                NEs.append(NE)
    return NEs

Я взял сэмпл из 30000 отзывов, чтобы побыстрее их обработать.

In [177]:
samp = dataset.sample(30000).dropna(subset=['reviewText'])

In [178]:
samp['NEs'] = samp['reviewText'].progress_apply(lambda r: NER(r))

100%|█████████████████████████████████████| 29975/29975 [12:37<00:00, 39.58it/s]


In [179]:
samp

,asin,reviewText,NEs
153277,B009AYLYWA,Well made and works as advertised.,[]
624289,B00QDBNCTG,This is the 2nd time I've had to replace my so...,"[son 's screen, correct tools, screen]"
382207,B00IKEOFCC,Excelent.,[]
942354,B01BHEIIR4,Does what it's supposed to do. Charges my phon...,"[phone, cord]"
255883,B00D856NOG,Love this. I leave this on the dining table fo...,"[phone, phone]"
...,...,...,...
431598,B00K35JGPC,Excellent product. I cannot complain whatsoev...,"[product, screen protector, tempered glass, ..."
732272,B00X5RV14Y,Very good battery use it for hiking works gre...,[charge]
781825,B010MWGM0W,This is a great Case for the Note 5. It allows...,"[case, note 5, needed grip, charger]"
713016,B00VU5YS0W,"Loved this phone case while I had an Iphone 5,...","[phone case, case, phone]"


In [357]:
samp['reviewText'] = samp['reviewText'].apply(lambda t: t.lower())

## Коллокации

In [358]:
stops = stopwords.words('english')

Я взял топ-10 наиболее частотных однословных сущностей для последующего анализа (обратите внимание, насколько "широкие" (*product*) и "узкие" (*iphone*) сущности сюда попали.

In [384]:
targets

['phone',
 'case',
 'product',
 'screen',
 'protection',
 'battery',
 'charger',
 'cases',
 'iphone',
 'phones']

In [359]:
targets = [s for t in samp['NEs'].to_list() for s in t if ' ' not in s]
targets = [pair[0] for pair in Counter(targets).most_common()[:10]]

In [360]:
tokenizer = RegexpTokenizer(r'\w+')
texts = samp['reviewText'].apply(tokenizer.tokenize).to_list()

При подборе биграмм я поставил 3 фильтра:
* второе слово в биграмме должно быть из списка сущностей (топ-10), о котором я писал выше;
* частотный фильтр — 10;
* фильтр стоп-слов.

In [361]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(texts)
finder.apply_ngram_filter(lambda w1, w2: w2 not in targets)
finder.apply_freq_filter(10)
finder.apply_word_filter(lambda w: w in stops)

### Log-likelihood

На мой взгляд, для этой задачи (и соответствующего отчёта) лучше всего подходит именно **log-likelihood**. Именно эта метра "поднимает" больше всего оценочных (типичных и важных для отзывов) биграмм (например, *great case*), а не биграмм по типу товара (например, *cell phone*).

In [362]:
finder.score_ngrams(bigram_measures.likelihood_ratio)[:15]

[(('great', 'product'), 3514.173321017227),
 (('glass', 'screen'), 2870.3291030645146),
 (('cell', 'phone'), 2691.684186097711),
 (('good', 'product'), 1454.1276982087452),
 (('wall', 'charger'), 1369.0988806171529),
 (('hook', 'product'), 1258.278883108626),
 (('excellent', 'product'), 1176.7946625967838),
 (('great', 'case'), 1051.00006909802),
 (('external', 'battery'), 996.6059884244687),
 (('car', 'charger'), 910.358596233918),
 (('good', 'protection'), 907.5703756959222),
 (('protective', 'case'), 808.8977944791238),
 (('nice', 'case'), 778.6208299121639),
 (('great', 'protection'), 777.3600606775939),
 (('touch', 'screen'), 766.4180806040268)]

### Pointwise mutual information (PMI)

In [363]:
finder.score_ngrams(bigram_measures.pmi)[:15]

[(('gp', 'product'), 8.0929775352122),
 (('external', 'battery'), 7.878939691682092),
 (('maximum', 'protection'), 7.870642661694454),
 (('wall', 'charger'), 7.85764018104345),
 (('adequate', 'protection'), 7.823669998234223),
 (('extended', 'battery'), 7.740940651856228),
 (('smart', 'phones'), 7.670134612111195),
 (('hook', 'product'), 7.51726904339068),
 (('portable', 'charger'), 7.335173606056056),
 (('cell', 'phones'), 7.246732069265242),
 (('rechargeable', 'battery'), 7.2257118575469335),
 (('wireless', 'charger'), 7.143549053129828),
 (('moderate', 'protection'), 7.126982414954238),
 (('rapid', 'charger'), 7.1136401874486985),
 (('spare', 'battery'), 6.846392098771236)]

### Jaccard index

In [364]:
finder.score_ngrams(bigram_measures.jaccard)[:15]

[(('great', 'product'), 0.056071918330032),
 (('wall', 'charger'), 0.05385466617484323),
 (('glass', 'screen'), 0.052370585389453314),
 (('cell', 'phones'), 0.04275286757038582),
 (('car', 'charger'), 0.03847258110824002),
 (('good', 'product'), 0.032452960469532194),
 (('wireless', 'charger'), 0.032153179190751446),
 (('excellent', 'product'), 0.03159383841559258),
 (('external', 'battery'), 0.029945292254534985),
 (('smart', 'phones'), 0.029265255292652552),
 (('portable', 'charger'), 0.027330587794833397),
 (('hook', 'product'), 0.02557352388115833),
 (('drop', 'protection'), 0.024719841793012523),
 (('good', 'protection'), 0.023839114224381425),
 (('android', 'phones'), 0.021645021645021644)]

## Группировка и вывод

In [381]:
output_products = ['product', 'charger', 'case', 'iphone', 'phone']
best = finder.score_ngrams(bigram_measures.likelihood_ratio)
results = {}
for t in targets:
    if not t in results:
        results[t] = []
    for b in best:
        if b[0][1] == t:
            results[t].append(b[0][0])

In [383]:
for out in output_products:
    print('out', '---', sep='\n')
    for colloc in results[out][:5]:
        print('{} {}'.format(colloc, out))
    print('...')

out
---
great product
good product
hook product
excellent product
quality product
...
out
---
wall charger
car charger
wireless charger
portable charger
usb charger
...
out
---
great case
protective case
nice case
tpu case
phone case
...
out
---
new iphone
gold iphone
apple iphone
white iphone
black iphone
...
out
---
cell phone
new phone
smart phone
windows phone
case phone
...
